In [191]:
import utils
import pandas as pd
import geopandas as gpd

In [192]:
df = pd.read_csv('data/Transactions.csv')
df.shape

(1609883, 46)

In [193]:
df.head()

,transaction_id,procedure_id,trans_group_id,trans_group_ar,trans_group_en,procedure_name_ar,procedure_name_en,instance_date,property_type_id,property_type_ar,...,rooms_en,has_parking,procedure_area,actual_worth,meter_sale_price,rent_value,meter_rent_price,no_of_parties_role_1,no_of_parties_role_2,no_of_parties_role_3
0,3-9-2006-163,9,3,هبات,Gifts,هبه,Grant,16-10-2006,4,فيلا,...,NaN,0,3162.42,12000000.0,3794.56,NaN,NaN,3.0,1.0,0.0
1,3-9-2019-2944,9,3,هبات,Gifts,هبه,Grant,13-11-2019,1,أرض,...,NaN,0,209.09,916659.0,4384.04,NaN,NaN,2.0,4.0,0.0
2,2-13-2001-690,13,2,رهون,Mortgages,تسجيل رهن,Mortgage Registration,20-08-2001,2,مبنى,...,NaN,0,1337.80,4519342.0,3378.19,NaN,NaN,1.0,1.0,0.0
3,2-13-2020-9477,13,2,رهون,Mortgages,تسجيل رهن,Mortgage Registration,30-11-2020,2,مبنى,...,NaN,0,278.71,2500000.0,8969.90,NaN,NaN,1.0,1.0,0.0
4,2-13-1999-532,13,2,رهون,Mortgages,تسجيل رهن,Mortgage Registration,26-04-1999,4,فيلا,...,NaN,0,3626.93,1900000.0,523.86,NaN,NaN,1.0,1.0,0.0


In [194]:
gdf_polygons = gpd.read_file("data/dubai.geojson")

In [195]:
gdf_polygons.head()

,CNAME_E,CNAME_A,COMMUNITY_E,COMMUNITY_A,COMM_NUM,SHAPE_AREA,SHAPE_LEN,Sector,Population 2018,Population 2019,Area Sq Km,Latitude,Longitude,geometry
0,HEFAIR,حفير,HEFAIR - 991,حفير - 991,991,143566502.005,51721.2111825,9,0,0,143.565131,24.693237,55.249140,"POLYGON ((55.24246 24.74379, 55.26266 24.74935..."
1,AL QUSAIS IND. SECOND,القصيص الصناعية الثانية,AL QUSAIS IND. SECOND - 243,القصيص الصناعية الثانية - 243,243,1728332.8155,6052.3652815,2,8063,8834,1.728331,25.281654,55.393184,"POLYGON ((55.3983 25.29, 55.39917 25.28948, 55..."
2,AL JAFILIYA,الجافلية,AL JAFILIYA - 323,الجافلية - 323,323,1669479.4215,5220.96603607,3,21957,23963,1.669479,25.237635,55.286851,"POLYGON ((55.28809 25.24352, 55.28829 25.24341..."
3,JABAL ALI SECOND,جبل علي الثانية,JABAL ALI SECOND - 592,جبل علي الثانية - 592,592,5059596.0639,10664.7225181,5,1561,1570,5.059545,25.053979,55.115282,"POLYGON ((55.13269 25.06475, 55.12714 25.0584,..."
4,WADI AL SAFA 5,وادي الصفا 5,WADI AL SAFA 5 - 648,وادي الصفا 5 - 648,648,16319666.8572,22350.6627618,6,19550,21241,16.318810,25.077814,55.352154,"POLYGON ((55.37906 25.10325, 55.37908 25.10322..."


In [196]:
gdf_polygons["CNAME_E"].value_counts().max()

np.int64(1)

In [197]:
# Convert the key columns to uppercase
gdf_polygons["CNAME_E_UP"] = gdf_polygons["CNAME_E"].str.upper()
df["area_name_en"] = df["area_name_en"].str.upper()

# Merge using uppercase columns
merged = df.merge(
    gdf_polygons[["CNAME_E_UP", "geometry"]],
    left_on="area_name_en",
    right_on="CNAME_E_UP",
    how="left"
)

merged.shape

(1609883, 48)

In [198]:
len(merged[merged["CNAME_E_UP"].isna()]["area_name_en"].unique())

151

In [199]:
len(merged[merged["CNAME_E_UP"].notna()]["area_name_en"].unique())

106

In [200]:
unique_addresses = df["area_name_en"].dropna().unique()
unique_addresses = [addr + ", Dubai, UAE" for addr in unique_addresses]

len(unique_addresses)

257

In [88]:
geo_lookup = {addr: utils.geocode_with_quality(addr) for addr in unique_addresses}

In [201]:
df["lat"] = df["area_name_en"].map(lambda a: geo_lookup.get(a + ", Dubai, UAE", {}).get("lat"))
df["lng"] = df["area_name_en"].map(lambda a: geo_lookup.get(a + ", Dubai, UAE", {}).get("lng"))
df["location_type"] = df["area_name_en"].map(lambda a: geo_lookup.get(a + ", Dubai, UAE", {}).get("location_type"))

In [202]:
mask = df[["lat","lng"]].isna().any(axis=1)
df[mask]["area_name_en"].value_counts()

Series([], Name: count, dtype: int64)

In [203]:
df["location_type"].value_counts()

location_type
APPROXIMATE         1487359
GEOMETRIC_CENTER     112897
ROOFTOP                9627
Name: count, dtype: int64

In [204]:
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["lng"], df["lat"]),
    crs="EPSG:4326"  # WGS84, matches your GeoJSON CRS84
)

# Make sure both are in same CRS
gdf_polygons = gdf_polygons.to_crs(gdf.crs)

df = gpd.sjoin(
    gdf,
    gdf_polygons[["CNAME_E", "geometry"]],
    how="left",
    predicate="within"
)

print(df.shape)

(1609883, 52)


In [205]:
""" df = df.rename(
      columns={"geometry": "point_geometry"}
    ).join(
    gdf_polygons[["geometry"]],
    on="index_right"
)

df = df.drop(columns=["index_right","point_geometry"], errors="ignore")

print(df.shape) """

df = df.drop(columns=["index_right", "geometry"], errors="ignore")

In [206]:
len(df[df["CNAME_E"].notna()]["area_name_en"].unique())

251

In [207]:
len(df[df["CNAME_E"].isna()]["area_name_en"].unique())

6

In [208]:
df[df["CNAME_E"].isna()]["area_name_en"].value_counts()

area_name_en
SAIH SHUAIB 2                 2735
UM SUQAIM FIRST               1133
SAIH SHUAIB 4                  655
SAIH SHUAIB 3                  273
AL QUSAIS INDUSTRIAL THIRD      46
AL-DZAHIYYAH AL-JADEEDAH         1
Name: count, dtype: int64

In [209]:
manual_map = {
    "SAIH SHUAIB 2":"SAIH SHUAIB 2",
    "SAIH SHUAIB 3":"SAIH SHUAIB 3",
    "SAIH SHUAIB 4":"SAIH SHUAIB 4",
    "UM SUQAIM FIRST":"UMM SUQEIM FIRST",
    "AL QUSAIS INDUSTRIAL THIRD":"AL QUSAIS IND. THIRD"
}

gdf_lut = gdf_polygons.set_index("CNAME_E")#[["COMMUNITY_E", "geometry"]]

# 3. Mask rows in df that we want to correct
mask = df["area_name_en"].isin(manual_map.keys())

# 4. Compute the correct CNAME_E for those rows
correct_cname = df.loc[mask, "area_name_en"].map(manual_map)

# 5. Assign CNAME_E
df.loc[mask, "CNAME_E"] = correct_cname

# 6. Assign COMMUNITY_E and polygon_geometry from geojson lookup
""" df.loc[mask, "COMMUNITY_E"] = correct_cname.map(gdf_lut["COMMUNITY_E"])
df.loc[mask, "geometry"] = correct_cname.map(gdf_lut["geometry"]) """

' df.loc[mask, "COMMUNITY_E"] = correct_cname.map(gdf_lut["COMMUNITY_E"])\ndf.loc[mask, "geometry"] = correct_cname.map(gdf_lut["geometry"]) '

In [210]:
len(df[df["CNAME_E"].notna()]["area_name_en"].unique())

256

In [211]:
len(df[df["CNAME_E"].isna()]["area_name_en"].unique())

1

In [212]:
df = df[df["CNAME_E"].notna()]

In [214]:
df = df.drop(columns=["location_type"], errors="ignore")

In [215]:
df.head()

,transaction_id,procedure_id,trans_group_id,trans_group_ar,trans_group_en,procedure_name_ar,procedure_name_en,instance_date,property_type_id,property_type_ar,...,actual_worth,meter_sale_price,rent_value,meter_rent_price,no_of_parties_role_1,no_of_parties_role_2,no_of_parties_role_3,lat,lng,CNAME_E
0,3-9-2006-163,9,3,هبات,Gifts,هبه,Grant,16-10-2006,4,فيلا,...,12000000.0,3794.56,NaN,NaN,3.0,1.0,0.0,25.245513,55.294135,MANKHOOL
1,3-9-2019-2944,9,3,هبات,Gifts,هبه,Grant,13-11-2019,1,أرض,...,916659.0,4384.04,NaN,NaN,2.0,4.0,0.0,25.245513,55.294135,MANKHOOL
2,2-13-2001-690,13,2,رهون,Mortgages,تسجيل رهن,Mortgage Registration,20-08-2001,2,مبنى,...,4519342.0,3378.19,NaN,NaN,1.0,1.0,0.0,25.239126,55.312967,OUD METHA
3,2-13-2020-9477,13,2,رهون,Mortgages,تسجيل رهن,Mortgage Registration,30-11-2020,2,مبنى,...,2500000.0,8969.90,NaN,NaN,1.0,1.0,0.0,25.224727,55.268742,AL BADA'
4,2-13-1999-532,13,2,رهون,Mortgages,تسجيل رهن,Mortgage Registration,26-04-1999,4,فيلا,...,1900000.0,523.86,NaN,NaN,1.0,1.0,0.0,25.224727,55.268742,AL BADA'


In [ ]:
""" df = df.set_geometry("geometry")
df.to_file("data/transactions.geojson", driver="GeoJSON") """

In [217]:
df.to_parquet("data/transactions.parquet", index=False)